In [1]:
titanic = spark.read.format('csv').option('header','true').option('inferSchema','true').load('/FileStore/tables/train_titanic.csv')
display(titanic.limit(10))

PassengerId,Survived,Pclass,_c3,Name,ff,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
1,0,3,null,Mr. Owen Harris,null,male,22.0,1,0,A/5 21171,7.25,null,S
2,1,1,null,Mrs. John Bradley (Florence Briggs Thayer),null,female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,null,Miss. Laina,null,female,26.0,0,0,STON/O2. 3101282,7.925,null,S
4,1,1,null,Mrs. Jacques Heath (Lily May Peel),null,female,35.0,1,0,113803,53.1,C123,S
5,0,3,null,Mr. William Henry,null,male,35.0,0,0,373450,8.05,null,S
6,0,3,null,Mr. James,null,male,null,0,0,330877,8.4583,null,Q
7,0,1,null,Mr. Timothy J,null,male,54.0,0,0,17463,51.8625,E46,S
8,0,3,null,Master. Gosta Leonard,null,male,2.0,3,1,349909,21.075,null,S
9,1,3,null,Mrs. Oscar W (Elisabeth Vilhelmina Berg),null,female,27.0,0,2,347742,11.1333,null,S
10,1,2,null,Mrs. Nicholas (Adele Achem),null,female,14.0,1,0,237736,30.0708,null,C


In [2]:
titanic = titanic.drop('_c3','_c13')

In [3]:
titanic.printSchema()

root
-- PassengerId: integer (nullable = true)
-- Survived: integer (nullable = true)
-- Pclass: integer (nullable = true)
-- Name: string (nullable = true)
-- ff: string (nullable = true)
-- Sex: string (nullable = true)
-- Age: double (nullable = true)
-- SibSp: integer (nullable = true)
-- Parch: integer (nullable = true)
-- Ticket: string (nullable = true)
-- Fare: double (nullable = true)
-- Cabin: string (nullable = true)
-- Embarked: string (nullable = true)

In [4]:
titanic.count()

Out[5]: 891

In [5]:
display(titanic.limit(10))


PassengerId,Survived,Pclass,Name,ff,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
1,0,3,Mr. Owen Harris,null,male,22.0,1,0,A/5 21171,7.25,null,S
2,1,1,Mrs. John Bradley (Florence Briggs Thayer),null,female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,Miss. Laina,null,female,26.0,0,0,STON/O2. 3101282,7.925,null,S
4,1,1,Mrs. Jacques Heath (Lily May Peel),null,female,35.0,1,0,113803,53.1,C123,S
5,0,3,Mr. William Henry,null,male,35.0,0,0,373450,8.05,null,S
6,0,3,Mr. James,null,male,null,0,0,330877,8.4583,null,Q
7,0,1,Mr. Timothy J,null,male,54.0,0,0,17463,51.8625,E46,S
8,0,3,Master. Gosta Leonard,null,male,2.0,3,1,349909,21.075,null,S
9,1,3,Mrs. Oscar W (Elisabeth Vilhelmina Berg),null,female,27.0,0,2,347742,11.1333,null,S
10,1,2,Mrs. Nicholas (Adele Achem),null,female,14.0,1,0,237736,30.0708,null,C


In [6]:
survived_analysis = titanic.select('Survived','PassengerId')
display(survived_analysis.limit(10))

Survived,PassengerId
0,1
1,2
1,3
1,4
0,5
0,6
0,7
0,8
1,9
1,10


In [7]:
from pyspark.sql.functions import col

count_of_passengers = survived_analysis.groupBy('Survived').count()
count_of_passengers = count_of_passengers.select('*',col("Survived").alias("binary representation of survival")).drop('Survived')
display(count_of_passengers)

count,binary representation of survival
342,1
549,0


In [8]:
new_dataset = spark.read.format('csv').option('header','true').option('inferSchema','true').load('/FileStore/tables/train_titanic.csv')
new_dataset.printSchema()

root
-- PassengerId: integer (nullable = true)
-- Survived: integer (nullable = true)
-- Pclass: integer (nullable = true)
-- _c3: string (nullable = true)
-- Name: string (nullable = true)
-- ff: string (nullable = true)
-- Sex: string (nullable = true)
-- Age: double (nullable = true)
-- SibSp: integer (nullable = true)
-- Parch: integer (nullable = true)
-- Ticket: string (nullable = true)
-- Fare: double (nullable = true)
-- Cabin: string (nullable = true)
-- Embarked: string (nullable = true)

In [9]:
display(new_dataset.limit(5))

PassengerId,Survived,Pclass,_c3,Name,ff,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
1,0,3,null,Mr. Owen Harris,null,male,22.0,1,0,A/5 21171,7.25,null,S
2,1,1,null,Mrs. John Bradley (Florence Briggs Thayer),null,female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,null,Miss. Laina,null,female,26.0,0,0,STON/O2. 3101282,7.925,null,S
4,1,1,null,Mrs. Jacques Heath (Lily May Peel),null,female,35.0,1,0,113803,53.1,C123,S
5,0,3,null,Mr. William Henry,null,male,35.0,0,0,373450,8.05,null,S


In [10]:
new_dataset = new_dataset.drop('_c3','ff')
new_dataset.printSchema()

root
-- PassengerId: integer (nullable = true)
-- Survived: integer (nullable = true)
-- Pclass: integer (nullable = true)
-- Name: string (nullable = true)
-- Sex: string (nullable = true)
-- Age: double (nullable = true)
-- SibSp: integer (nullable = true)
-- Parch: integer (nullable = true)
-- Ticket: string (nullable = true)
-- Fare: double (nullable = true)
-- Cabin: string (nullable = true)
-- Embarked: string (nullable = true)

In [11]:
new_dataset.dropDuplicates().count()

Out[46]: 891

In [12]:
from pyspark.sql.functions import when, count, col
new_dataset = new_dataset.select([count(when(col(c).isNull(), c)).alias(c) for c in 
           new_dataset.columns])
display(new_dataset)

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,0,0,0,0,177,0,0,0,0,687,2


Under Age column , we can see - 177 rows has no data. So we will first start with gender analysis.

In [14]:
new_dataset.agg({'Age':'min'}).withColumnRenamed("min(Age)", "minimum_age").show() , new_dataset.agg({'Age':'max'}).withColumnRenamed("max(Age)", "maximum_age").show()


+-----------+
minimum_age|
+-----------+
 0.92|
+-----------+

+-----------+
maximum_age|
+-----------+
 80.0|
+-----------+

Out[43]: (None, None)

In [15]:
new_dataset = new_dataset.groupBy('Sex').count()
display(new_dataset)

Sex,count
female,314
male,577


In [16]:
new_dataset.printSchema()

root
-- PassengerId: integer (nullable = true)
-- Survived: integer (nullable = true)
-- Pclass: integer (nullable = true)
-- Name: string (nullable = true)
-- Sex: string (nullable = true)
-- Age: double (nullable = true)
-- SibSp: integer (nullable = true)
-- Parch: integer (nullable = true)
-- Ticket: string (nullable = true)
-- Fare: double (nullable = true)
-- Cabin: string (nullable = true)
-- Embarked: string (nullable = true)

In [17]:
from pyspark.sql.functions import col

new_dataset[new_dataset.Sex.isin("female") & new_dataset.Survived.isin(1)].count() , new_dataset[new_dataset.Sex.isin("male") & new_dataset.Survived.isin(1)].count()

Out[29]: (233, 109)

so after the calculations , we can say that the number of female survivor(233) is greater than the number of male survivor(109) though the total count of male passenger(577) is more.

In [19]:
display(new_dataset.limit(7))

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
1,0,3,Mr. Owen Harris,male,22.0,1,0,A/5 21171,7.25,null,S
2,1,1,Mrs. John Bradley (Florence Briggs Thayer),female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,Miss. Laina,female,26.0,0,0,STON/O2. 3101282,7.925,null,S
4,1,1,Mrs. Jacques Heath (Lily May Peel),female,35.0,1,0,113803,53.1,C123,S
5,0,3,Mr. William Henry,male,35.0,0,0,373450,8.05,null,S
6,0,3,Mr. James,male,null,0,0,330877,8.4583,null,Q
7,0,1,Mr. Timothy J,male,54.0,0,0,17463,51.8625,E46,S


In [20]:
new_dataset.agg({'Fare':'min'}).show() , new_dataset.agg({'Fare':'max'}).show()

+---------+
min(Fare)|
+---------+
 0.0|
+---------+

+---------+
max(Fare)|
+---------+
 512.3292|
+---------+

Out[31]: (None, None)

In [21]:
display(new_dataset[new_dataset.Fare.isin(0.0)])

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
180,0,3,Mr. Lionel,male,36.0,0,0,LINE,0.0,null,S
264,0,1,Mr. William,male,40.0,0,0,112059,0.0,B94,S
272,1,3,Mr. William Henry,male,25.0,0,0,LINE,0.0,null,S
278,0,2,Mr. Francis,male,null,0,0,239853,0.0,null,S
303,0,3,Mr. William Cahoone Jr,male,19.0,0,0,LINE,0.0,null,S
414,0,2,Mr. Alfred Fleming,male,null,0,0,239853,0.0,null,S
467,0,2,Mr. William,male,null,0,0,239853,0.0,null,S
482,0,2,Mr. Anthony Wood,male,null,0,0,239854,0.0,null,S
598,0,3,Mr. Alfred,male,49.0,0,0,LINE,0.0,null,S
634,0,1,Mr. William Henry Marsh,male,null,0,0,112052,0.0,null,S


so we can see here - 
1. 15 people out of 891 paid no money to board the ship. 
2. All 15 were MALE 
3. 14 of them didnt have a cabin.
3. None of them survived except only 1 person.

In [23]:
display(new_dataset[new_dataset.Fare.isin(512.3292)])

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
259,1,1,Miss. Anna,female,35.0,0,0,PC 17755,512.3292,null,C
680,1,1,Mr. Thomas Drake Martinez,male,36.0,0,1,PC 17755,512.3292,B51 B53 B55,C
738,1,1,Mr. Gustave J,male,35.0,0,0,PC 17755,512.3292,B101,C


In [24]:
from pyspark.sql.functions import *

new_dataset = new_dataset.select('*',explode(split(col("Cabin"), " ")).alias('CabinDetails')).drop('Cabin')
display(new_dataset.limit(10))

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,CabinDetails
2,1,1,Mrs. John Bradley (Florence Briggs Thayer),female,38.0,1,0,PC 17599,71.2833,C,C85
4,1,1,Mrs. Jacques Heath (Lily May Peel),female,35.0,1,0,113803,53.1,S,C123
7,0,1,Mr. Timothy J,male,54.0,0,0,17463,51.8625,S,E46
11,1,3,Miss. Marguerite Rut,female,4.0,1,1,PP 9549,16.7,S,G6
12,1,1,Miss. Elizabeth,female,58.0,0,0,113783,26.55,S,C103
22,1,2,Mr. Lawrence,male,34.0,0,0,248698,13.0,S,D56
24,1,1,Mr. William Thompson,male,28.0,0,0,113788,35.5,S,A6
28,0,1,Mr. Charles Alexander,male,19.0,3,2,19950,263.0,S,C23
28,0,1,Mr. Charles Alexander,male,19.0,3,2,19950,263.0,S,C25
28,0,1,Mr. Charles Alexander,male,19.0,3,2,19950,263.0,S,C27


In [25]:
display(new_dataset[new_dataset.Fare.isin(512.3292)])

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,CabinDetails
680,1,1,Mr. Thomas Drake Martinez,male,36.0,0,1,PC 17755,512.3292,C,B51
680,1,1,Mr. Thomas Drake Martinez,male,36.0,0,1,PC 17755,512.3292,C,B53
680,1,1,Mr. Thomas Drake Martinez,male,36.0,0,1,PC 17755,512.3292,C,B55
738,1,1,Mr. Gustave J,male,35.0,0,0,PC 17755,512.3292,C,B101


In [26]:
new_dataset.count()

Out[41]: 238

So we can analyse here - 
1. Out of total 891 rows, Cabin had only 204 rows with data in it. (687 null values).
2. After exploding the Cabin column (multiple values) total count of rows is - 238.
3. 34 new rows have been created after exploding.

In [28]:
from pyspark.sql.functions import *

new_dataset.filter(col("Age").between(0.4,18)).count()

Out[47]: 139

In [29]:
from pyspark.sql.functions import *

minimum_age = new_dataset.select('PassengerId','Pclass','Survived','Name','Sex','Age').filter((col("Age").between(0.4,18)) & (col("Survived") == 1))
minimum_age.count()

Out[48]: 70

In [30]:
display(minimum_age.limit(5))

PassengerId,Pclass,Survived,Name,Sex,Age
10,2,1,Mrs. Nicholas (Adele Achem),female,14.0
11,3,1,Miss. Marguerite Rut,female,4.0
23,3,1,Miss. Anna,female,15.0
40,3,1,Miss. Jamila,female,14.0
44,2,1,Miss. Simonne Marie Anne Andree,female,3.0


In [31]:
group_Pclass = minimum_age.groupBy('Pclass').count()
display(group_Pclass)

Pclass,count
1,14
3,33
2,23
